## A convenient way to inquire Simbad

Using this notebook, you can get the position and velocity of a star in Heliocentric Mean Ecliptic Cartesian coordinates by entering its name.

In [1]:
from astroquery.simbad import Simbad
from astropy.coordinates import SkyCoord
import astropy.units as u
import numpy as np



In [2]:
stellar_name = 'HD 7977' # 'Gliese 710'

In [3]:
custom_simbad = Simbad()
custom_simbad.add_votable_fields('parallax', 'rv_value', 
                                'pmra', 'pmdec')

result = custom_simbad.query_object(str(stellar_name))

if result is None:
    print("Cannot find the data of" + str(stellar_name))
else:
    # Extract RA, Dec, parallax, radial velocity (rv_value), and proper motion (pmra, pmdec)
    ra = result['RA'][0]  # 'hh mm ss'
    dec = result['DEC'][0]  # 'dd mm ss'
    parallax = result['PLX_VALUE'][0]  # milliarcseconds (mas)
    rv = result['RV_VALUE'][0] * u.km / u.s  # radial velocity (km/s)
    pm_ra = result['PMRA'][0] * u.mas / u.yr  # proper motion in RA direction (mas/yr)
    pm_dec = result['PMDEC'][0] * u.mas / u.yr   # proper motion in Dec direction (mas/yr)

    # Calculate distance: parallax (mas) to distance (pc)
    distance = (1000 / parallax) * u.pc

    # Calculate total proper motion (angular velocity)
    proper_motion = np.sqrt(pm_ra**2 + pm_dec**2)  # mas/yr

    # Create SkyCoord object, including radial velocity and distance
    stellar = SkyCoord(ra=ra, dec=dec, unit=(u.hourangle, u.deg), 
                    distance=distance, pm_ra_cosdec=pm_ra, pm_dec=pm_dec, 
                    radial_velocity=rv, frame='icrs')
    
    # Transform to heliocentric mean ecliptic
    stellar_helio = stellar.transform_to('heliocentricmeanecliptic')
    
    # Convert position and velocity to Cartesian coordinates
    pos = stellar_helio.cartesian.xyz.to(u.AU)
    vel = stellar_helio.velocity.d_xyz.to(u.km/u.s)
    
    print(f"The position of " + str(stellar_name) + " in Heliocentric Mean Ecliptic Cartesian coordinates (x, y, z):")
    print(pos)
    print(f"The velocity of " + str(stellar_name) + " in Heliocentric Mean Ecliptic Cartesian coordinates (vx, vy, vz):")
    print(vel)



The position of HD 7977 in Heliocentric Mean Ecliptic Cartesian coordinates (x, y, z):
[ 6908192.15169089  7800658.4339682  11626104.95499581] AU
The velocity of HD 7977 in Heliocentric Mean Ecliptic Cartesian coordinates (vx, vy, vz):
[11.67373463 13.27281649 19.67970122] km / s
